In [52]:
library(mimi)
library(missMDA)
library(reticulate)
library(ROCR)
np <- import("numpy")

In [59]:
mca.logi.fn <- function(Y, R, ncp=4, maxit=100){
    Y[R==0] <- NA
    Y <- as.data.frame(Y)
    nC <- dim(Y)[2]
    for (i in 1:nC){
        Y[, i] <- as.factor(Y[, i])
    }
    res <- imputeMCA(Y, ncp=ncp, maxit=maxit, method="EM")
    return(res)
}

post.mca.fn <- function(res){
    probs <- res$tab.disj
    probs[probs<=0] = 0
    probs[probs>=1] = 1
    idxs <- seq(2, dim(probs)[2], 2)
}

In [54]:
mimi.logi.fn <- function(Y, X, R, CT, Cb, alp.init, theta.init, trace.it=T, maxit=100){
  p <- ncol(X)
  m <- dim(Y)[1]
  n <- dim(Y)[2]
  var.type <- rep("binomial", n)
  Y.miss <- Y
  Y.miss[R==0] <- NA
  res <- mimi(Y.miss, model="c", x=X, var.type=var.type, lambda1=CT, lambda2 = Cb, 
              alpha0=alp.init, theta0=theta.init, algo="bc", maxit=maxit, trace.it = trace.it, max.rank=10)
  return(res)
}

post.mimi.fn <- function(res){
    probs <-  exp(res$param)/(1+exp(res$param))
    return(probs)
}

In [90]:
probs2auc <- function(probs, Ynp, rYnp, Rnp){
    idxMat <- rYnp != -1 & Rnp == 0
    obs <- Ynp[idxMat]
    eProbs <- probs[idxMat]
    pred <- prediction(eProbs, obs)
    auc.res <- performance(pred, "auc")
    return(auc.res@y.values)
}

In [124]:
aucs2Idxs <- function(aucs){
    Cb.idx <- ifelse(which.max(aucs) %% 3==0, which.max(aucs) %/% 3, which.max(aucs) %/% 3 + 1)
    CT.idx <- ifelse(which.max(aucs) %% 3==0, 3, which.max(aucs) %% 3)
    return(c(Cb.idx, CT.idx))
}

In [56]:
Ynp <- np$load("./npData/Y.npz")
rYnp <- np$load("./npData/rY.npz")
Xnp <- np$load("./npData/X.npz")
Rnp <- np$load("./npData/R_50_1.npz")

In [57]:
m = dim(Xnp)[1]
n = dim(Xnp)[2]
p = dim(Xnp)[3]
alp.init <- rep(0, p)
theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
matX <- matrix(Xnp, nrow=m*n)

In [63]:
res <- mimi.logi.fn(Ynp, matX, Rnp, CT=1, Cb=1, alp.init=alp.init, theta.init=theta.init)

[1] "iter  1 : error  1  - objective:  153426.409720027"
[1] "iter  2 : error  0.852542145118306  - objective:  22623.9292595152"
[1] "iter  3 : error  0.0756587473728733  - objective:  20912.2311110878"
[1] "iter  4 : error  1.73964164214063e-16  - objective:  20912.2311110878"


In [91]:
probs <- post.mimi.fn(res)
probs2auc(probs, Ynp, rYnp, Rnp)

[[1]]
[1] 0.5724635

In [92]:
Rnp.ps <- dir("./npData/.", pattern="R_50.*npz", full.name=1)
Rnp0 <- np$load(Rnp.ps[1])

In [99]:
Ynp <- Ynp[1:30, 1:40]
Xnp <- Xnp[1:30, 1:40, ]
Rnp0 <- Rnp0[1:30, 1:40]
rYnp <- rYnp[1:30, 1:40]

In [103]:
m = dim(Xnp)[1]
n = dim(Xnp)[2]
p = dim(Xnp)[3]
alp.init <- rep(0, p)
theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
matX <- matrix(Xnp, nrow=m*n)

In [101]:
CTs <- c(1, 1e2, 1e3)
Cbs <- c(1, 1e2, 1e3)

In [104]:
alp.init <- rep(0, p)
theta.init <- matrix(rnorm(n*m), nrow=m) + 0.1
aucs <- c()
for (Cb in Cbs){
    for (CT in CTs){
        res <- mimi.logi.fn(Ynp, matX, Rnp0, CT=CT, Cb=Cb, alp.init=alp.init, theta.init=theta.init)
        probs <- post.mimi.fn(res)
        cur.auc <- probs2auc(probs, Ynp, rYnp, Rnp0) 
        aucs <- c(aucs, cur.auc)
    }
}

[1] "iter  1 : error  1  - objective:  368.223383328066"
[1] "iter  2 : error  0.444972590378452  - objective:  204.374070610659"
[1] "iter  3 : error  0.00542153991039408  - objective:  203.266048430193"
[1] "iter  4 : error  2.79650336589927e-16  - objective:  203.266048430193"
[1] "iter  1 : error  1  - objective:  368.223383328066"
[1] "iter  2 : error  46.1653973485353  - objective:  17367.4021876902"
[1] "iter  3 : error  0  - objective:  17367.4021876902"
[1] "iter  1 : error  1  - objective:  368.223383328066"
[1] "iter  2 : error  469.790901046871  - objective:  173356.218423547"
[1] "iter  3 : error  0  - objective:  173356.218423547"
[1] "iter  1 : error  1  - objective:  368.223383328066"
[1] "iter  2 : error  0.444972590378452  - objective:  204.374070610659"
[1] "iter  3 : error  0.00542153991039408  - objective:  203.266048430193"
[1] "iter  4 : error  2.79650336589927e-16  - objective:  203.266048430193"
[1] "iter  1 : error  1  - objective:  368.223383328066"
[1] "iter

In [126]:
idxs <- aucs2Idxs(aucs)
selCb <- Cbs[idxs[1]]
selCT <- CTs[idxs[2]]

In [130]:
fName <- paste0("./", "MIMI_", OR, ".RData")